### Semantic Search with FAISS

In [2]:
# load dataset
from datasets import load_dataset


df_install = load_dataset("carlosug/ResearchInstall", split="train")
df_install



Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['software', 'repo_name', 'readme_url', 'content', 'plan', 'steps', 'optional_steps', 'extra_info_optional'],
    num_rows: 39
})

In [3]:
columns = df_install.column_names
columns
columns_to_keep = ["software", "content", "readme_url", "steps", "plan"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
df_install = df_install.remove_columns(columns_to_remove)
df_install

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 39
})

In [4]:
df_install.to_pandas()

,software,readme_url,content,plan,steps
0,vcr-video-representation-for-contextual,https://raw.githubusercontent.com/oronnir/VCR/...,None,None,None
1,ensuring-trustworthy-and-ethical-behaviour-in,https://raw.githubusercontent.com/AAAI-DISIM-U...,## Installation\n\n**OS X & Linux:**\n1. To d...,"binary, source",[plan binary]>>step1. follow the instructions ...
2,synthesizing-sentiment-controlled-feedback,https://raw.githubusercontent.com/MIntelligenc...,None,None,None
3,only-the-curve-shape-matters-training,https://raw.githubusercontent.com/cfeng783/GTT...,## Getting Started\n\n#### Install dependencie...,source,[plan source]>> [INCOMPLETE] step1. Install de...
4,from-uncertainty-to-precision-enhancing,https://raw.githubusercontent.com/fer-agathe/c...,None,None,None
5,stochastic-gradient-flow-dynamics-of-test,https://raw.githubusercontent.com/rodsveiga/sg...,None,None,None
6,accuracy-of-textfooler-black-box-adversarial,https://raw.githubusercontent.com/zero-one-los...,None,None,None
7,differentially-private-decentralized-learning-1,https://raw.githubusercontent.com/totilas/DPra...,None,None,None
8,aydiv-adaptable-yielding-3d-object-detection,https://raw.githubusercontent.com/sanjay-810/A...,### **Installation**\n1. Prepare for the runn...,"source,docker",[plan source]>> step1. Prepare for the running...
9,cartesian-atomic-cluster-expansion-for,https://raw.githubusercontent.com/BingqingChen...,## Installation\n\nPlease refer to the `setup....,source,[plan source]>>[INCOMPLETE] step1. please refe...


In [5]:
df_install = df_install.filter(
    lambda x: (x["content"] != None)
)
df_install

Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 29
})

In [6]:
df_install.set_format("pandas")
df2install = df_install[:]
# df2install["readme_url"][0].tolist() error str object has no attribute 'tolist'

In [7]:
df_install.set_format("pandas")
dataset_carlos = df_install[:]
dataset_carlos["steps"][2]

'[plan source]>> step1. Prepare for the running environment. step2. prepare for the data:```cd Aydiv python depth_to_lidar.py ```\n[plan docker]>> step1. You can use the docker image provided by [`OpenPCDet`](https://github.com/open-mmlab/OpenPCDet)'

In [26]:
content_df = df2install.explode("content", ignore_index=True)
content_df.head(4)
content_df.dtypes

software      object
readme_url    object
content       object
plan          object
steps         object
dtype: object

In [20]:
from datasets import Dataset

content_dataset = Dataset.from_pandas(content_df)
content_dataset

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 29
})

In [21]:
content_dataset = content_dataset.map(
    lambda x: {"content_length": len(x["content"].split())} # to count number of rows in a cell comments
)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [22]:
# content_dataset = content_dataset.filter(lambda x: x["content_length"] > 1) # filter small comments
# content_dataset

In [28]:
def concatenate_text(examples):
    return {
        "text": examples["software"]
        + " \n "
        + examples["content"]
        + " \n "
        + examples["plan"]
        + " \n "
        + examples["steps"]
        + " \n "
        # + examples["seq_order"] no string
    }


content_dataset = content_dataset.map(concatenate_text)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [29]:
content_dataset["text"]



['ensuring-trustworthy-and-ethical-behaviour-in \n ## Installation\n\n**OS X & Linux:**\n1.  To download and install SICStus Prolog (it is needed), follow the instructions at https://sicstus.sics.se/download4.html.\n2.  Then, you can download DALI and test it by running an example DALI MAS:\n```sh\ngit clone https://github.com/AAAI-DISIM-UnivAQ/DALI.git\ncd DALI/Examples/advanced\nbash startmas.sh\n```\n&nbsp;&nbsp;&nbsp;&nbsp; You will see different windows opening:\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog LINDA server (active_server_wi.pl)\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog FIPA client (active_user_wi.pl) \n* &nbsp;&nbsp;&nbsp;&nbsp; 1 instance of DALI metaintepreter for each agent (active_dali_wi.pl)\n\n**Windows:**\n1.  To download and install SICStus Prolog (it is needed), follow the instructions at https://sicstus.sics.se/download4.html.\n2.  Then, you can download DALI from https://github.com/AAAI-DISIM-UnivAQ/DALI.git.\n3.  Unzip the repository, go to the folder "DALI/Examples/basic"

In [30]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [31]:
import torch

device = torch.device("cpu")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [32]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [33]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [34]:
# from the dataset
embedding = get_embeddings(content_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [35]:
embeddings_dataset = content_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
# !conda install faiss-cpu -y # algorithms to quickly search and cluster embedding vectors it needs to be installed via conda

In [ ]:
# !conda install -c pytorch faiss-cpu -y

In [156]:
# import faiss-cpu conda
# embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████| 3/3 [00:00<00:00, 340.74it/s]


Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [36]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps', 'content_length', 'text', 'embeddings'],
    num_rows: 29
})

In [44]:
question = 'what is the plan of https://raw.githubusercontent.com/cfeng783/GTT?'
question_embedding = get_embeddings([question]).cpu().detach().numpy()

question_embedding.shape

(1, 768)

In [45]:
scores, samples = embeddings_dataset.get_nearest_examples("embeddings", question_embedding, k=5)

In [46]:
import pandas as pd

samples_df= pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=False)

,software,readme_url,content,plan,steps,content_length,text,embeddings,scores
4,only-the-curve-shape-matters-training,https://raw.githubusercontent.com/cfeng783/GTT...,## Getting Started\n\n#### Install dependencie...,source,[plan source]>> [INCOMPLETE] step1. Install de...,15,only-the-curve-shape-matters-training \n ## Ge...,"[-0.25307658314704895, -0.6750497221946716, -0...",48.823574
3,genegpt-teaching-large-language-models-to-use,https://raw.githubusercontent.com/ncbi/GeneGPT...,# Requirements\n\nThe code has been tested wit...,source,[plan source]>>step1.install requirements:\n``...,104,genegpt-teaching-large-language-models-to-use ...,"[-0.4030008614063263, -0.32260626554489136, -0...",47.592957
2,geometric-slosh-free-tracking-for-robotic,https://raw.githubusercontent.com/jonarriza96/...,## Installation\n\n### Dependencies\n\nInitial...,source,[plan source]>> step1. Check dependencies. ste...,69,geometric-slosh-free-tracking-for-robotic \n #...,"[-0.3325411379337311, -0.45105645060539246, -0...",44.597107
1,guiding-instruction-based-image-editing-via,https://raw.githubusercontent.com/apple/ml-mgi...,## Requirements\n```\nconda create -n mgie pyt...,source,[plan source]>> step1. create conda environmen...,121,guiding-instruction-based-image-editing-via \n...,"[-0.19101731479167938, -0.4127528667449951, -0...",43.995201
0,tensorflow,https://raw.githubusercontent.com/tensorflow/t...,## Install\n\nSee the [TensorFlow install guid...,packagemanager,[plan packagemanager]>>via pip. step1.:\n```\n...,95,tensorflow \n ## Install\n\nSee the [TensorFlo...,"[-0.4453235864639282, -0.4245409369468689, -0....",43.752434


In [47]:
for _, row in samples_df.iterrows():
    print(f"CONTENT: {row.content}")
    print(f"SCORE: {row.scores}")
    print(f"SOFTWARE: {row.software}")
    print(f"METHOD: {row.plan}")
    print(f"STEPS: {row.steps}")
    print("=" * 50)
    print()

CONTENT: ## Install

See the [TensorFlow install guide](https://www.tensorflow.org/install) for the
[pip package](https://www.tensorflow.org/install/pip), to
[enable GPU support](https://www.tensorflow.org/install/gpu), use a
[Docker container](https://www.tensorflow.org/install/docker), and
[build from source](https://www.tensorflow.org/install/source).

To install the current release, which includes support for
[CUDA-enabled GPU cards](https://www.tensorflow.org/install/gpu) *(Ubuntu and
Windows)*:

```
$ pip install tensorflow
```

Other devices (DirectX and MacOS-metal) are supported using
[Device plugins](https://www.tensorflow.org/install/gpu_plugins#available_devices).

A smaller CPU-only package is also available:

```
$ pip install tensorflow-cpu
```

To update TensorFlow to the latest version, add `--upgrade` flag to the above
commands.

*Nightly binaries are available for testing using the
[tf-nightly](https://pypi.python.org/pypi/tf-nightly) and
[tf-nightly-cpu](https://pyp